In [0]:
import torch, torchvision, matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
torch.manual_seed(0)

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
np.random.seed(0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [0]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [0]:
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Mutate. For every 100th training data, 
        # Mislabel
        labels[0] = (labels[0]+1)%num_classes 
        # Duplicate
        images[1], labels[1] = images[2], labels[2]
        # Remove
        images = images[:-1].to(device)
        labels = labels[:-1].to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1,
                           num_epochs,
                           int((i+1)*len(outputs)/batch_size),
                           int(total_step-(total_step/batch_size)),
                           loss.item()))


Epoch [1/5], Step [99/594], Loss: 0.1941
Epoch [1/5], Step [198/594], Loss: 0.1974
Epoch [1/5], Step [297/594], Loss: 0.1927
Epoch [1/5], Step [396/594], Loss: 0.1580
Epoch [1/5], Step [495/594], Loss: 0.1858
Epoch [1/5], Step [594/594], Loss: 0.1687
Epoch [2/5], Step [99/594], Loss: 0.1308
Epoch [2/5], Step [198/594], Loss: 0.1185
Epoch [2/5], Step [297/594], Loss: 0.1354
Epoch [2/5], Step [396/594], Loss: 0.0885
Epoch [2/5], Step [495/594], Loss: 0.1713
Epoch [2/5], Step [594/594], Loss: 0.1179
Epoch [3/5], Step [99/594], Loss: 0.1008
Epoch [3/5], Step [198/594], Loss: 0.1235
Epoch [3/5], Step [297/594], Loss: 0.1283
Epoch [3/5], Step [396/594], Loss: 0.1372
Epoch [3/5], Step [495/594], Loss: 0.0906
Epoch [3/5], Step [594/594], Loss: 0.1502
Epoch [4/5], Step [99/594], Loss: 0.1393
Epoch [4/5], Step [198/594], Loss: 0.1135
Epoch [4/5], Step [297/594], Loss: 0.1257
Epoch [4/5], Step [396/594], Loss: 0.0904
Epoch [4/5], Step [495/594], Loss: 0.1180
Epoch [4/5], Step [594/594], Loss: 0.0

In [0]:
path = F"/content/gdrive/My Drive/correct_test_dataset.txt"
correct_test = np.loadtxt(path, delimiter=",", dtype=int)

In [0]:
# Dictionary to record which indices are incorrectly classified
# incorrection_dict[true_label] = what model thinks the class is
incorrection_dict = {}
incorrection_dict[0] = []
incorrection_dict[1] = []
incorrection_dict[2] = []
incorrection_dict[3] = []
incorrection_dict[4] = []
incorrection_dict[5] = []
incorrection_dict[5] = []
incorrection_dict[6] = []
incorrection_dict[7] = []
incorrection_dict[8] = []
incorrection_dict[9] = []

In [0]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        #only test ones correctly classified by the original model
        for j in range(len(predicted)):
          if idx*batch_size+j in correct_test:
            if labels[j] == predicted[j]:
              correct +=1
            else:
              incorrection_dict[labels[j].item()].append(predicted[j].item())

    total = len(correct_test)
        
    print('Test Accuracy of the model on the {} test images (100% for original model): {} %'.format(total, 100 * correct / total))


Test Accuracy of the model on the 9896 test images (100% for original model): 99.36337914308811 %


In [0]:
print(correct/total)

0.9936337914308812


In [0]:
incorrection_dict

{0: [],
 1: [3, 2],
 2: [7, 7, 0, 0, 0, 8, 7],
 3: [5, 8, 5, 5, 2, 7, 5, 7, 8, 5, 7, 8, 5, 7, 5],
 4: [2, 9, 7, 9, 2, 6, 0, 9, 9],
 5: [8],
 6: [0, 8, 1, 1, 8, 1, 0, 8, 1, 0, 7],
 7: [4, 2, 9, 9, 2, 1],
 8: [5],
 9: [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 7]}

In [0]:
path =F"/content/gdrive/My Drive/correction_dict.npy" 
correction_dict=np.load(path, allow_pickle=True)

In [0]:
def calc_mutation_score():
  score = 0
  for key, value in incorrection_dict.items():
    count_misclassify = len(value)
    normalizing = len(correction_dict.item().get(key))
    score += count_misclassify/normalizing
  return score
calc_mutation_score()

0.0636079528751964